## **1. 读取数据与因子计算**

"""
**数据准备模块**
用于分析碳排放变化的驱动因素分解
"""

In [1]:
import pandas as pd
import numpy as np

# 1. 读取数据
file_path = '数据.xlsx'

df_fc = pd.read_excel(file_path, sheet_name='FC（2010不变价）')      # 最终消费数据
df_pop = pd.read_excel(file_path, sheet_name='人口')                 # 人口数据
df_energy = pd.read_excel(file_path, sheet_name='能源消费总量')      # 能源消费数据
df_carbon = pd.read_excel(file_path, sheet_name='碳排放量')          # 碳排放数据

# 2. 数据整合
data = pd.DataFrame()
data['年份'] = df_fc['年份']
data['FC'] = df_fc['GDP']  # GDP列实际是FC数据
data['POP'] = df_pop['人口总量']
data['PE'] = df_energy['能源消费总量']
data['CC'] = df_carbon['真实消费端碳排放量']

# 3. 计算LMDI分解因子
# 根据LMDI方法，将碳排放分解为四个因子的乘积：CC = m × s × g × p
data['m'] = data['CC'] / data['PE']  # 碳强度因子
data['s'] = data['PE'] / data['FC']  # 能源结构因子
data['g'] = data['FC'] / data['POP']  # 人均消费因子
data['p'] = data['POP']  # 人口因子

print("\nLMDI因子计算：")
print("   m = CC/PE (碳强度因子)")
print("   s = PE/FC (能源结构因子)") 
print("   g = FC/POP (人均消费因子)")
print("   p = POP (人口因子)")

# 4. 验证恒等式 CC = m × s × g × p
data['CC_verify'] = data['m'] * data['s'] * data['g'] * data['p']
verification_error = np.abs(data['CC'] - data['CC_verify']).max()
print(f"\n恒等式验证: 最大误差 = {verification_error:.6f}")

# 5. 数据展示
display_years = [1997, 2000, 2005, 2010, 2015, 2020, 2021]
display_data = data[data['年份'].isin(display_years)].copy()

# 设置数据显示格式
pd.set_option('display.float_format', lambda x: '%.4f' % x)

print("\n原始数据：")
print(display_data[['年份', 'CC', 'PE', 'FC', 'POP']].to_string(index=False))
print("\n计算的因子：")
print(display_data[['年份', 'm', 's', 'g', 'p']].to_string(index=False))
print("\n数据准备完成，可进行LMDI分解计算")


LMDI因子计算：
   m = CC/PE (碳强度因子)
   s = PE/FC (能源结构因子)
   g = FC/POP (人均消费因子)
   p = POP (人口因子)

恒等式验证: 最大误差 = 0.000000

原始数据：
  年份         CC     PE          FC    POP
1997  3014.4642 135909 119587.6239 123626
2000  2921.9404 146964 150922.2288 126743
2005  4736.6050 261369 241242.4944 130756
2010  7439.1033 360648 412119.2558 134091
2015  8788.3347 434113 603767.1409 138326
2020  9688.9137 498314 799062.4118 141212
2021 10156.2255 525896 867781.7792 141260

计算的因子：
  年份      m      s      g      p
1997 0.0222 1.1365 0.9673 123626
2000 0.0199 0.9738 1.1908 126743
2005 0.0181 1.0834 1.8450 130756
2010 0.0206 0.8751 3.0734 134091
2015 0.0202 0.7190 4.3648 138326
2020 0.0194 0.6236 5.6586 141212
2021 0.0193 0.6060 6.1432 141260

数据准备完成，可进行LMDI分解计算


## **2. LMDI算法实现（成功复现）**

"""
**算法实现模块**
实现LMDI加法分解方法，用于分析碳排放变化的驱动因素
"""

### **2.1. 函数定义**

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def log_mean(a, b):
    """
    计算对数平均值 L(a,b)
    
    L(a,b)是LMDI方法的核心，用于权重计算
    
    当a≈b时，使用a作为近似值；否则使用 (a-b)/(ln(a)-ln(b))

    参数:
    a, b: 两个正数
    
    返回: 对数平均值
    
    """
    if np.abs(a - b) < 1e-10:   # 当两个值非常接近时，避免数值不稳定
        return a
    else:
        return (a - b) / (np.log(a) - np.log(b))


def lmdi_single_decomposition(cc0, cc1, m0, m1, s0, s1, g0, g1, p0, p1):
    """
    执行单期LMDI加法分解
    
    将碳排放变化分解为四个效应的加法贡献：ΔCC = Δm + Δs + Δg + Δp
    
    参数:
    cc0, cc1: 基期和当期的碳排放量
    m0, m1: 基期和当期的碳强度因子 (CC/PE)
    s0, s1: 基期和当期的能源强度因子 (PE/FC)
    g0, g1: 基期和当期的人均消费因子 (FC/POP)
    p0, p1: 基期和当期的人口规模因子
    
    返回:
    (delta_m, delta_s, delta_g, delta_p): 各因素对碳排放变化的贡献值
    
    """
    # 数据有效性检查
    if any(v <= 0 for v in [cc0, cc1, m0, m1, s0, s1, g0, g1, p0, p1]):
        print(f"警告：检测到非正值数据，可能影响分解结果的准确性")
    
    L = log_mean(cc1, cc0)       # 对数平均权重
    
    delta_m = L * np.log(m1/m0)  # 碳强度效应
    delta_s = L * np.log(s1/s0)  # 能源结构效应
    delta_g = L * np.log(g1/g0)  # 消费规模效应
    delta_p = L * np.log(p1/p0)  # 人口规模效应
    
    return delta_m, delta_s, delta_g, delta_p


def lmdi_yearly_decomposition(data, start_year, end_year, return_results=True):
    """
    LMDI逐年分解
    
    对指定时期内的碳排放变化进行逐年分解，并计算各因素的累积贡献百分比
    
    参数:
    data: 包含年份、CC、m、s、g、p等列的数据框
    start_year: 开始年份（包含）
    end_year: 结束年份（包含）
    return_results: 是否返回结果DataFrame，默认False（仅打印）
    
    返回:
    DataFrame or None
        如果return_results=True，返回包含分解结果的DataFrame
        否则仅打印结果，返回None
    """
    # 提取指定年份范围的数据
    years = range(start_year, end_year + 1)
    data_period = data[data['年份'].isin(years)].copy()
    data_period = data_period.reset_index(drop=True)
    
    # 存储逐年分解结果
    results = []
    
    # 逐年计算LMDI分解
    for i in range(len(data_period)-1):
        year0 = int(data_period.loc[i, '年份'])
        year1 = int(data_period.loc[i+1, '年份'])
        
        # 获取相邻两年的数据
        cc0 = data_period.loc[i, 'CC']
        cc1 = data_period.loc[i+1, 'CC']
        
        # 调用lmdi_single_decomposition函数执行LMDI分解计算
        delta_m, delta_s, delta_g, delta_p = lmdi_single_decomposition(
            cc0, cc1,
            data_period.loc[i, 'm'], data_period.loc[i+1, 'm'],
            data_period.loc[i, 's'], data_period.loc[i+1, 's'],
            data_period.loc[i, 'g'], data_period.loc[i+1, 'g'],
            data_period.loc[i, 'p'], data_period.loc[i+1, 'p']
        )
        
        # 保存年度分解结果
        results.append({
            '年份': f"{year0}-{year1}",
            '碳强度效应': delta_m,
            '能源强度效应': delta_s,  
            '人均消费效应': delta_g,  
            '人口规模效应': delta_p,
            '碳排放变化': cc1 - cc0
        })

    df_results = pd.DataFrame(results)
    
    # 计算整个时期的累积效应
    cumulative = {
        '碳强度效应': df_results['碳强度效应'].sum(),
        '能源强度效应': df_results['能源强度效应'].sum(),
        '人均消费效应': df_results['人均消费效应'].sum(),
        '人口规模效应': df_results['人口规模效应'].sum()
    }
    
    # 计算碳排放总变化量
    total_change = sum(cumulative.values())
    
    # 输出分解结果
    print(f"{'-'*50}")
    print(f"{start_year}-{end_year}年 LMDI分解：\n")
    
    for effect, value in cumulative.items():
        if abs(total_change) > 1e-10:  # 避免除零
            contribution = value / abs(total_change) * 100
            sign = "+" if value > 0 else ""
            # 用颜色标识正负贡献
            print(f"{effect:14} : {sign}{contribution:6.2f}%")
        else:
            print(f"{effect:14} : 0.00%")

    print(f"{'合计':14} : {100.00 if abs(total_change) > 1e-10 else 0:7.2f}%")

    print(f"\n碳排放总变化量: {total_change:.2f} 万吨CO2")
    print(f"{'-'*50}")
    print('\n')

    # 根据参数决定是否返回结果
    if return_results:
        # 添加累积结果到DataFrame
        cumulative_row = {
            '年份': f"{start_year}-{end_year}",
            '碳强度效应': cumulative['碳强度效应'],
            '能源强度效应': cumulative['能源强度效应'],
            '人均消费效应': cumulative['人均消费效应'],
            '人口规模效应': cumulative['人口规模效应'],
            '碳排放变化': total_change
        }
        df_results = pd.concat([df_results, pd.DataFrame([cumulative_row])], ignore_index=True)
        
        # 添加贡献率列
        for effect in ['碳强度效应', '能源强度效应', '人均消费效应', '人口规模效应']:
            df_results[f'{effect}_贡献率%'] = df_results[effect] / abs(total_change) * 100
        
        return df_results
    else:
        return None

### **2.2. 执行LMDI分解**

In [3]:
# 分析各个五年计划期间
results_1 = lmdi_yearly_decomposition(data, 2000, 2005)  # "十五"期间
results_2 = lmdi_yearly_decomposition(data, 2005, 2010)  # "十一五"期间
results_3 = lmdi_yearly_decomposition(data, 2010, 2015)  # "十二五"期间
results_4 = lmdi_yearly_decomposition(data, 2015, 2020)  # "十三五"期间

# 分析整个研究期间
results_5 = lmdi_yearly_decomposition(data, 1997, 2021)  # 全时期分析

--------------------------------------------------
2000-2005年 LMDI分解：

碳强度效应          : -20.65%
能源强度效应         : + 24.45%
人均消费效应         : + 89.96%
人口规模效应         : +  6.24%
合计             :  100.00%

碳排放总变化量: 1814.66 万吨CO2
--------------------------------------------------


--------------------------------------------------
2005-2010年 LMDI分解：

碳强度效应          : + 30.68%
能源强度效应         : -46.60%
人均消费效应         : +110.42%
人口规模效应         : +  5.50%
合计             :  100.00%

碳排放总变化量: 2702.50 万吨CO2
--------------------------------------------------


--------------------------------------------------
2010-2015年 LMDI分解：

碳强度效应          : -13.41%
能源强度效应         : -124.12%
人均消费效应         : +218.14%
人口规模效应         : + 19.39%
合计             :  100.00%

碳排放总变化量: 1349.23 万吨CO2
--------------------------------------------------


--------------------------------------------------
2015-2020年 LMDI分解：

碳强度效应          : -42.81%
能源强度效应         : -144.91%
人均消费效应         : +266.64%
人口规模效应         : + 21

## **3. 基于增长率扰动的蒙特卡洛模拟**

### **3.1. 不确定性分析**

"""
**蒙特卡洛不确定性分析模块**
通过增长率扰动的蒙特卡洛模拟评估LMDI分解结果的稳健性
"""

#### **3.1.1. 函数定义**

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


# 数据扰动生成函数
def generate_perturbed_series_growth(data_original, growth_uncertainty, use_uniform_for_pop=True):
    """
    基于增长率扰动生成保持趋势的时间序列
    
    该方法通过对原始数据的年度增长率添加随机扰动，生成符合不确定性分布的新时间序列，同时保持整体趋势
    
    参数:
    data_original: 
        原始数据，包含CC、FC、PE、POP等列
    growth_uncertainty:
        各变量增长率的相对扰动强度
        - CC、FC、PE: 正态分布的标准差
        - POP: 均匀分布的半宽度（如0.08表示±8%的均匀分布）
    use_uniform_for_pop: 
        是否对人口使用均匀分布（默认True）
    
    返回:
    扰动后的完整数据（包括重新计算的LMDI因子）
    
    """
    n_years = len(data_original)
    data_perturbed = data_original.copy()

    # 对每个核心变量进行增长率扰动
    for var in ['CC', 'FC', 'PE', 'POP']:
        # 获取第一年的基准值（不扰动）
        base_value = data_original.loc[0, var]
        perturbed_values = [base_value]
        
        # 逐年计算扰动后的值
        for i in range(n_years - 1):
            if data_original.loc[i, var] > 0:
                # 计算原始年度增长率
                original_growth = (data_original.loc[i+1, var] - 
                                 data_original.loc[i, var]) / data_original.loc[i, var]
                
                # 对增长率施加相对扰动
                if var == 'POP' and use_uniform_for_pop:
                    # 人口使用均匀分布 U(1-std, 1+std)
                    growth_multiplier = np.random.uniform(
                        1 - growth_uncertainty[var], 
                        1 + growth_uncertainty[var]
                    )
                else:
                    # 其他变量使用正态分布 N(1, std)
                    growth_multiplier = np.random.normal(1, growth_uncertainty[var])
                
                perturbed_growth = original_growth * growth_multiplier
                
                # 基于扰动后的增长率计算下一年的值
                next_value = perturbed_values[-1] * (1 + perturbed_growth)
                # 确保数值为正（避免计算错误）
                perturbed_values.append(max(next_value, 0.001))
            else:
                # 如果原始值为0或负，保持原值
                perturbed_values.append(data_original.loc[i+1, var])
        
        data_perturbed[var] = perturbed_values
    
    # 重新计算LMDI分解因子
    data_perturbed['m'] = data_perturbed['CC'] / data_perturbed['PE']   # 碳强度因子
    data_perturbed['s'] = data_perturbed['PE'] / data_perturbed['FC']   # 能源强度因子
    data_perturbed['g'] = data_perturbed['FC'] / data_perturbed['POP']  # 人均消费因子
    data_perturbed['p'] = data_perturbed['POP']                         # 人口规模因子
    
    return data_perturbed


# 蒙特卡洛主函数
def lmdi_monte_carlo_growth(data, start_year, end_year, n_simulations=10000,
                            cc_growth_std=0.15, fc_growth_std=0.10, 
                            pe_growth_std=0.12, pop_growth_std=0.08,
                            random_seed=2025):
    """
    基于增长率扰动的LMDI蒙特卡洛不确定性分析
    
    通过大量模拟评估LMDI分解结果对数据不确定性的敏感性，提供各因素贡献率的统计分布和置信区间
    
    参数:
    data: 原始数据
    start_year: 分析起始年份（包含）
    end_year: 分析结束年份（包含）
    n_simulations: 蒙特卡洛模拟次数，默认10000次
    cc_growth_std: 碳排放增长率的扰动强度（正态分布标准差），默认0.15
    fc_growth_std: 最终消费增长率的扰动强度（正态分布标准差），默认0.10
    pe_growth_std: 能源消费增长率的扰动强度（正态分布标准差），默认0.12
    pop_growth_std: 人口增长率的扰动强度（均匀分布半宽度），默认0.08
    random_seed: 随机种子，用于结果可重现
    
    返回:
    包含各因素统计结果的字典
        - mean: 平均贡献率
        - std: 标准差
        - ci_low: 95%置信区间下限
        - ci_high: 95%置信区间上限
    """
    # 设置随机种子以确保结果可重现
    if random_seed is not None:
        np.random.seed(random_seed)

    # 设定各变量的增长率扰动参数
    growth_uncertainty = {
        'CC': cc_growth_std,
        'FC': fc_growth_std,
        'PE': pe_growth_std,
        'POP': pop_growth_std  # 注意：POP将使用均匀分布
    }
    
    # 提取指定时期的原始数据
    years = range(start_year, end_year + 1)
    data_original = data[data['年份'].isin(years)].copy()
    data_original = data_original.reset_index(drop=True)
    
    # 初始化结果存储
    results_all = {
        '碳强度效应': [],
        '能源强度效应': [],     # 修正名称
        '人均消费效应': [],     # 修正名称
        '人口规模效应': []
    }
    
    # 执行蒙特卡洛模拟
    for sim in tqdm(range(n_simulations), desc="模拟进度"):
        # 生成扰动后的时间序列
        data_perturbed = generate_perturbed_series_growth(data_original, growth_uncertainty)
        
        # 初始化累积效应
        cumulative = {
            '碳强度效应': 0,
            '能源强度效应': 0,
            '人均消费效应': 0,
            '人口规模效应': 0
        }
        
        # 逐年进行LMDI分解
        for j in range(len(data_perturbed)-1):
            # 获取相邻两年的数据
            cc0 = data_perturbed.loc[j, 'CC']
            cc1 = data_perturbed.loc[j+1, 'CC']
            
            # 确保数据有效性
            if cc0 > 0 and cc1 > 0:
                # 执行单期LMDI分解
                delta_m, delta_s, delta_g, delta_p = lmdi_single_decomposition(
                    cc0, cc1,
                    data_perturbed.loc[j, 'm'], data_perturbed.loc[j+1, 'm'],
                    data_perturbed.loc[j, 's'], data_perturbed.loc[j+1, 's'],
                    data_perturbed.loc[j, 'g'], data_perturbed.loc[j+1, 'g'],
                    data_perturbed.loc[j, 'p'], data_perturbed.loc[j+1, 'p']
                )
                
                # 累积各效应
                cumulative['碳强度效应'] += delta_m
                cumulative['能源强度效应'] += delta_s
                cumulative['人均消费效应'] += delta_g
                cumulative['人口规模效应'] += delta_p
        
        # 计算贡献百分比
        total_change = sum(cumulative.values())
        
        if abs(total_change) > 1e-10:  # 避免除零错误
            for effect in results_all.keys():
                contribution_pct = cumulative[effect] / abs(total_change) * 100
                results_all[effect].append(contribution_pct)
    
    # 统计分析与结果输出
    # 首先展示基准分解结果（使用原始数据）
    print("\n【基准分解结果（使用原始数据）】")
    lmdi_yearly_decomposition(data, start_year, end_year)
    
    # 展示蒙特卡洛统计结果
    print("【蒙特卡洛模拟统计】")
    print("-" * 80)
    print(f"{'驱动因素':14} {'均值':>10} {'标准差':>10} {'95%置信区间':>25}")
    print("-" * 80)
    
    stats = {}
    for effect, values in results_all.items():
        values = np.array(values)
        # 计算统计指标
        mean_val = np.mean(values)
        std_val = np.std(values)
        ci_low = np.percentile(values, 2.5)   # 95%置信区间下限
        ci_high = np.percentile(values, 97.5)  # 95%置信区间上限
        
        # 保存统计结果
        stats[effect] = {
            'mean': mean_val,
            'std': std_val,
            'ci_low': ci_low,
            'ci_high': ci_high
        }
        
        # 格式化输出
        sign = "+" if mean_val > 0 else ""
        print(f"{effect:14} {sign}{mean_val:9.2f}% {std_val:9.2f}% [{ci_low:7.2f}%, {ci_high:7.2f}%]")
    
    print("-" * 80)
    
    # 输出模拟参数信息
    print(f"\n【模拟参数设置】")
    print(f"• 模拟次数：{n_simulations:,}次")
    print(f"• 增长率扰动设置：")
    print(f"  - 碳排放量(CC)：正态分布 N(1, {cc_growth_std}) - 相对标准差 ±{cc_growth_std*100:.0f}%")
    print(f"  - 最终消费(FC)：正态分布 N(1, {fc_growth_std}) - 相对标准差 ±{fc_growth_std*100:.0f}%")
    print(f"  - 能源消费(PE)：正态分布 N(1, {pe_growth_std}) - 相对标准差 ±{pe_growth_std*100:.0f}%")
    print(f"  - 人口(POP)：均匀分布 U({1-pop_growth_std:.2f}, {1+pop_growth_std:.2f}) - 范围 ±{pop_growth_std*100:.0f}%")
    
    return stats


### **3.1.2. 参数设置** 

In [5]:
# 设定合理的不确定性范围
RECOMMENDED_GROWTH_UNCERTAINTY = {
    'cc_growth_std': 0.25,   # 碳排放：正态分布，数据不确定性较大
    'fc_growth_std': 0.20,   # 经济数据：正态分布，有一定不确定性
    'pe_growth_std': 0.15,   # 能源统计：正态分布，相对准确
    'pop_growth_std': 0.10,  # 人口数据：均匀分布，最为可靠
    'random_seed': 2025        # 设置随机种子以确保可重现性
}

# 执行分时段不确定性分析
print("LMDI分解不确定性分析（蒙特卡洛模拟）")
print("\n说明：")
print("• CC、FC、PE使用正态分布扰动（反映测量误差的正态性）")
print("• POP使用均匀分布扰动（反映人口统计的系统性不确定性）")
print("• 设置随机种子确保结果可重现")

LMDI分解不确定性分析（蒙特卡洛模拟）

说明：
• CC、FC、PE使用正态分布扰动（反映测量误差的正态性）
• POP使用均匀分布扰动（反映人口统计的系统性不确定性）
• 设置随机种子确保结果可重现


### **3.1.3. 不确定性分析**

#### **'十五'期间 (2001-2005)**

In [6]:
stats_2000_2005 = lmdi_monte_carlo_growth(data, 2000, 2005, **RECOMMENDED_GROWTH_UNCERTAINTY)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 881.09it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
2000-2005年 LMDI分解：

碳强度效应          : -20.65%
能源强度效应         : + 24.45%
人均消费效应         : + 89.96%
人口规模效应         : +  6.24%
合计             :  100.00%

碳排放总变化量: 1814.66 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应             -22.43%     16.78% [ -60.10%,    5.58%]
能源强度效应         +    25.00%     12.48% [   1.45%,   50.20%]
人均消费效应         +    91.08%     14.07% [  66.63%,  122.22%]
人口规模效应         +     6.34%      0.80% [   5.06%,    8.22%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.25) - 相对标准差 ±25%
  - 最终消费(FC)：正态分布 N(1, 0.2) - 相对标准差 ±20%
  - 能源消费(PE)：正态分布 N(1, 0.15) - 相对标

#### **'十一五'期间 (2006-2010)**

In [7]:
stats_2005_2010 = lmdi_monte_carlo_growth(data, 2005, 2010, **RECOMMENDED_GROWTH_UNCERTAINTY)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 886.40it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
2005-2010年 LMDI分解：

碳强度效应          : + 30.68%
能源强度效应         : -46.60%
人均消费效应         : +110.42%
人口规模效应         : +  5.50%
合计             :  100.00%

碳排放总变化量: 2702.50 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应          +    29.80%      9.51% [   8.96%,   45.89%]
能源强度效应            -46.91%     12.66% [ -73.14%,  -23.82%]
人均消费效应         +   111.53%     16.62% [  82.81%,  147.22%]
人口规模效应         +     5.58%      0.66% [   4.50%,    7.10%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.25) - 相对标准差 ±25%
  - 最终消费(FC)：正态分布 N(1, 0.2) - 相对标准差 ±20%
  - 能源消费(PE)：正态分布 N(1, 0.15) - 相对标

#### **'十二五'期间 (2011-2015)**

In [8]:
stats_2010_2015 = lmdi_monte_carlo_growth(data, 2010, 2015, **RECOMMENDED_GROWTH_UNCERTAINTY)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 881.21it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
2010-2015年 LMDI分解：

碳强度效应          : -13.41%
能源强度效应         : -124.12%
人均消费效应         : +218.14%
人口规模效应         : + 19.39%
合计             :  100.00%

碳排放总变化量: 1349.23 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应             -15.72%     20.07% [ -61.93%,   15.00%]
能源强度效应           -125.84%     29.91% [-191.50%,  -75.23%]
人均消费效应         +   221.77%     40.27% [ 157.81%,  313.71%]
人口规模效应         +    19.79%      3.09% [  15.24%,   27.15%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.25) - 相对标准差 ±25%
  - 最终消费(FC)：正态分布 N(1, 0.2) - 相对标准差 ±20%
  - 能源消费(PE)：正态分布 N(1, 0.15) - 相对标准

#### **'十三五'期间 (2016-2020)**

In [9]:
stats_2015_2020 = lmdi_monte_carlo_growth(data, 2015, 2020, **RECOMMENDED_GROWTH_UNCERTAINTY)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:11<00:00, 880.39it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
2015-2020年 LMDI分解：

碳强度效应          : -42.81%
能源强度效应         : -144.91%
人均消费效应         : +266.64%
人口规模效应         : + 21.07%
合计             :  100.00%

碳排放总变化量: 900.58 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应             -44.50%     19.99% [ -88.63%,  -10.84%]
能源强度效应           -145.64%     33.70% [-216.88%,  -86.21%]
人均消费效应         +   268.80%     42.13% [ 196.55%,  360.18%]
人口规模效应         +    21.34%      2.66% [  17.02%,   27.46%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.25) - 相对标准差 ±25%
  - 最终消费(FC)：正态分布 N(1, 0.2) - 相对标准差 ±20%
  - 能源消费(PE)：正态分布 N(1, 0.15) - 相对标准差

#### **整个研究期间 (1997-2021)**

In [10]:
stats_1997_2021 = lmdi_monte_carlo_growth(data, 1997, 2021, **RECOMMENDED_GROWTH_UNCERTAINTY)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:32<00:00, 307.05it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
1997-2021年 LMDI分解：

碳强度效应          :  -6.96%
能源强度效应         : -63.40%
人均消费效应         : +159.31%
人口规模效应         : + 11.04%
合计             :  100.00%

碳排放总变化量: 7141.76 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应              -7.61%      7.78% [ -24.02%,    6.45%]
能源强度效应            -63.48%      8.56% [ -80.51%,  -47.30%]
人均消费效应         +   159.98%     11.70% [ 138.47%,  184.47%]
人口规模效应         +    11.11%      0.67% [   9.92%,   12.51%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.25) - 相对标准差 ±25%
  - 最终消费(FC)：正态分布 N(1, 0.2) - 相对标准差 ±20%
  - 能源消费(PE)：正态分布 N(1, 0.15) - 相对标准差

### **3.2. 敏感性分析**

"""
**敏感性分析模块**
通过单因素扰动分析评估各变量不确定性对LMDI分解结果的影响
"""

#### **3.2.1 函数定义**

In [11]:
def single_factor_sensitivity_analysis(data, start_year, end_year, perturbation_level=0.20, n_simulations=10000):
    """
    单因素敏感性分析
    
    逐个扰动每个变量，保持其他变量不变，评估各变量的不确定性对LMDI分解结果的影响程度
    
    参数:
    data: 原始数据
    start_year: 分析起始年份
    end_year: 分析结束年份
    perturbation_level: 扰动强度（所有变量使用相同强度），默认0.20 (±20%)
    n_simulations: 每个变量的模拟次数，默认10000
    
    返回:
    敏感性分析结果汇总表
    
    """
    print("\n" + "="*70)
    print(f"单因素敏感性分析 ({start_year}-{end_year})")
    print("="*70)
    print(f"\n扰动强度：±{perturbation_level*100:.0f}%")
    print(f"模拟次数：每个变量{n_simulations:,}次")
    print("-"*70)
    
    # 定义要分析的变量
    variables = ['CC', 'FC', 'PE', 'POP']
    variable_names = {
        'CC': '碳排放量',
        'FC': '最终消费',
        'PE': '能源消费',
        'POP': '人口'
    }
    
    # 存储敏感性分析结果
    sensitivity_results = []
    
    for var in variables:
        print(f"\n正在分析 {variable_names[var]} 的敏感性...")
        
        # 创建单因素扰动配置（只扰动当前变量）
        uncertainty_config = {
            'cc_growth_std': perturbation_level if var == 'CC' else 0.00,
            'fc_growth_std': perturbation_level if var == 'FC' else 0.00,
            'pe_growth_std': perturbation_level if var == 'PE' else 0.00,
            'pop_growth_std': perturbation_level if var == 'POP' else 0.00
        }
        
        # 执行蒙特卡洛模拟
        stats = lmdi_monte_carlo_growth(data, start_year, end_year, n_simulations=n_simulations, 
                                        **uncertainty_config)
        
        # 记录该变量扰动对各效应的影响
        for effect, values in stats.items():
            sensitivity_results.append({
                '扰动变量': variable_names[var],
                '影响效应': effect,
                '贡献率均值': values['mean'],
                '贡献率标准差': values['std'],
                '95%CI下限': values['ci_low'],
                '95%CI上限': values['ci_high'],
                '不确定性范围': values['ci_high'] - values['ci_low']
            })
    
    df_sensitivity = pd.DataFrame(sensitivity_results)
    
    return df_sensitivity

#### **3.2.2. 基础设置**

In [12]:
# 全时期敏感性分析（1997-2021）
print("全时期敏感性分析 (1997-2021)")

# 逐个变量进行敏感性分析
variables_to_analyze = [
    ('CC', '碳排放量', 0.20),
    ('FC', '最终消费', 0.20),
    ('PE', '能源消费', 0.20),
    ('POP', '人口', 0.20)
]

print("\n将依次分析以下变量的敏感性：")
for var, name, level in variables_to_analyze:
    if var == 'POP':
        print(f"  • {name}: 均匀分布 U({1-level:.2f}, {1+level:.2f}) - 范围 ±{level*100:.0f}%")
    else:
        print(f"  • {name}: 正态分布 N(1, {level}) - 标准差 ±{level*100:.0f}%")

全时期敏感性分析 (1997-2021)

将依次分析以下变量的敏感性：
  • 碳排放量: 正态分布 N(1, 0.2) - 标准差 ±20%
  • 最终消费: 正态分布 N(1, 0.2) - 标准差 ±20%
  • 能源消费: 正态分布 N(1, 0.2) - 标准差 ±20%
  • 人口: 均匀分布 U(0.80, 1.20) - 范围 ±20%


#### **3.2.3. 敏感性分析**

##### **变量1：碳排放**

In [13]:
SENSITIVITY_CC = {
    'cc_growth_std': 0.20,   # 仅扰动碳排放（正态分布）
    'fc_growth_std': 0.00,   
    'pe_growth_std': 0.00,   
    'pop_growth_std': 0.00,
    'random_seed': 2025
}
stats_cc = lmdi_monte_carlo_growth(data, 1997, 2021, **SENSITIVITY_CC)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:32<00:00, 307.26it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
1997-2021年 LMDI分解：

碳强度效应          :  -6.96%
能源强度效应         : -63.40%
人均消费效应         : +159.31%
人口规模效应         : + 11.04%
合计             :  100.00%

碳排放总变化量: 7141.76 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应              -7.40%      5.45% [ -18.78%,    2.44%]
能源强度效应            -63.59%      2.48% [ -68.76%,  -59.10%]
人均消费效应         +   159.91%      7.38% [ 146.59%,  175.27%]
人口规模效应         +    11.09%      0.52% [  10.15%,   12.18%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.2) - 相对标准差 ±20%
  - 最终消费(FC)：正态分布 N(1, 0.0) - 相对标准差 ±0%
  - 能源消费(PE)：正态分布 N(1, 0.0) - 相对标准差 ±0

##### **变量2：最终消费**

In [14]:
SENSITIVITY_FC = {
    'cc_growth_std': 0.00,   
    'fc_growth_std': 0.20,   # 仅扰动最终消费（正态分布）
    'pe_growth_std': 0.00,   
    'pop_growth_std': 0.00,
    'random_seed': 2025
}
stats_fc = lmdi_monte_carlo_growth(data, 1997, 2021, **SENSITIVITY_FC)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:32<00:00, 308.46it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
1997-2021年 LMDI分解：

碳强度效应          :  -6.96%
能源强度效应         : -63.40%
人均消费效应         : +159.31%
人口规模效应         : + 11.04%
合计             :  100.00%

碳排放总变化量: 7141.76 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应              -6.96%      0.00% [  -6.96%,   -6.96%]
能源强度效应            -63.11%      7.10% [ -77.05%,  -49.22%]
人均消费效应         +   159.02%      7.10% [ 145.14%,  172.96%]
人口规模效应         +    11.04%      0.00% [  11.04%,   11.04%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.0) - 相对标准差 ±0%
  - 最终消费(FC)：正态分布 N(1, 0.2) - 相对标准差 ±20%
  - 能源消费(PE)：正态分布 N(1, 0.0) - 相对标准差 ±0

##### **变量3：能源消费**

In [15]:
SENSITIVITY_PE = {
    'cc_growth_std': 0.00,   
    'fc_growth_std': 0.00,   
    'pe_growth_std': 0.20,   # 仅扰动能源消费（正态分布）
    'pop_growth_std': 0.00,
    'random_seed': 2025
}
stats_pe = lmdi_monte_carlo_growth(data, 1997, 2021, **SENSITIVITY_PE)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:32<00:00, 306.85it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
1997-2021年 LMDI分解：

碳强度效应          :  -6.96%
能源强度效应         : -63.40%
人均消费效应         : +159.31%
人口规模效应         : + 11.04%
合计             :  100.00%

碳排放总变化量: 7141.76 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应              -6.83%      4.76% [ -16.23%,    2.53%]
能源强度效应            -63.52%      4.76% [ -72.89%,  -54.12%]
人均消费效应         +   159.31%      0.00% [ 159.31%,  159.31%]
人口规模效应         +    11.04%      0.00% [  11.04%,   11.04%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.0) - 相对标准差 ±0%
  - 最终消费(FC)：正态分布 N(1, 0.0) - 相对标准差 ±0%
  - 能源消费(PE)：正态分布 N(1, 0.2) - 相对标准差 ±20

##### **变量4：人口**

In [16]:
SENSITIVITY_POP = {
    'cc_growth_std': 0.00,   
    'fc_growth_std': 0.00,   
    'pe_growth_std': 0.00,   
    'pop_growth_std': 0.20,  # 仅扰动人口（均匀分布）
    'random_seed': 2025
}
stats_pop = lmdi_monte_carlo_growth(data, 1997, 2021, **SENSITIVITY_POP)

模拟进度: 100%|█████████████████████████████████████████████████████████████████| 10000/10000 [00:32<00:00, 307.25it/s]


【基准分解结果（使用原始数据）】
--------------------------------------------------
1997-2021年 LMDI分解：

碳强度效应          :  -6.96%
能源强度效应         : -63.40%
人均消费效应         : +159.31%
人口规模效应         : + 11.04%
合计             :  100.00%

碳排放总变化量: 7141.76 万吨CO2
--------------------------------------------------


【蒙特卡洛模拟统计】
--------------------------------------------------------------------------------
驱动因素                   均值        标准差                   95%置信区间
--------------------------------------------------------------------------------
碳强度效应              -6.96%      0.00% [  -6.96%,   -6.96%]
能源强度效应            -63.40%      0.00% [ -63.40%,  -63.40%]
人均消费效应         +   159.32%      0.28% [ 158.76%,  159.87%]
人口规模效应         +    11.04%      0.28% [  10.49%,   11.59%]
--------------------------------------------------------------------------------

【模拟参数设置】
• 模拟次数：10,000次
• 增长率扰动设置：
  - 碳排放量(CC)：正态分布 N(1, 0.0) - 相对标准差 ±0%
  - 最终消费(FC)：正态分布 N(1, 0.0) - 相对标准差 ±0%
  - 能源消费(PE)：正态分布 N(1, 0.0) - 相对标准差 ±0%

## **4. 逐年分解与直接分解对比（不重要）**

"""
**方法对比与结果分析模块**
对比直接分解与逐年累加两种LMDI实施方法，分析碳排放变化的驱动因素
"""

In [17]:
import pandas as pd
import numpy as np

# ===========================
# LMDI核心算法
# ===========================

def log_mean(a, b):
    """
    计算对数平均值 L(a,b)
    
    参数:
    ----
    a, b : float
        两个正数值
    
    返回:
    ----
    float : 对数平均值
    """
    if np.abs(a - b) < 1e-10:
        return a
    else:
        return (a - b) / (np.log(a) - np.log(b))


def lmdi_additive_decomposition(data, start_year, end_year):
    """
    执行LMDI加法分解（单期）
    
    直接计算从start_year到end_year的分解效应
    
    参数:
    ----
    data : DataFrame
        包含年份和LMDI因子的数据
    start_year : int
        基准年
    end_year : int
        目标年
    
    返回:
    ----
    dict : 各效应的贡献值
    """
    # 获取基准年和目标年数据
    data_0 = data[data['年份'] == start_year].iloc[0]
    data_t = data[data['年份'] == end_year].iloc[0]
    
    # 提取变量值
    CC_0, CC_t = data_0['CC'], data_t['CC']
    m_0, m_t = data_0['m'], data_t['m']
    s_0, s_t = data_0['s'], data_t['s']
    g_0, g_t = data_0['g'], data_t['g']
    p_0, p_t = data_0['p'], data_t['p']
    
    # 计算对数平均权重
    L = log_mean(CC_t, CC_0)
    
    # 计算各效应贡献（加法分解）
    delta_CC_m = L * np.log(m_t / m_0)  # 碳强度效应
    delta_CC_s = L * np.log(s_t / s_0)  # 能源强度效应
    delta_CC_g = L * np.log(g_t / g_0)  # 人均消费效应
    delta_CC_p = L * np.log(p_t / p_0)  # 人口规模效应
    
    return {
        'delta_CC_m': delta_CC_m,
        'delta_CC_s': delta_CC_s,
        'delta_CC_g': delta_CC_g,
        'delta_CC_p': delta_CC_p,
        'delta_CC_total': CC_t - CC_0  # 实际总变化
    }

# ===========================
# 数据准备与基础分析
# ===========================

print("="*70)
print("LMDI分解方法对比分析：1997-2021年")
print("="*70)

# 获取基础数据
CC_1997 = data[data['年份']==1997]['CC'].values[0]
CC_2021 = data[data['年份']==2021]['CC'].values[0]
total_change = CC_2021 - CC_1997
growth_rate = (CC_2021/CC_1997 - 1) * 100

# 输出基础信息
print(f"\n【基础信息】")
print("-"*50)
print(f"• 1997年碳排放量: {CC_1997:,.2f} 万吨CO2")
print(f"• 2021年碳排放量: {CC_2021:,.2f} 万吨CO2")
print(f"• 总变化量: {total_change:,.2f} 万吨CO2")
print(f"• 累计增长率: {growth_rate:.2f}%")
print(f"• 年均增长率: {(np.power(CC_2021/CC_1997, 1/24) - 1)*100:.2f}%")

# ===========================
# 方法1：直接分解
# ===========================

print("\n" + "="*70)
print("方法1：直接分解法（1997→2021）")
print("="*70)
print("说明：直接计算整个时期的分解效应，反映长期累积影响")

# 执行直接分解
direct = lmdi_additive_decomposition(data, 1997, 2021)

# 计算贡献率
direct_contribution = {}
for key in ['delta_CC_m', 'delta_CC_s', 'delta_CC_g', 'delta_CC_p']:
    direct_contribution[key] = (direct[key] / abs(total_change)) * 100

# 输出结果表格
print(f"\n{'效应类型':<15} {'贡献量(万吨CO2)':>15} {'贡献率(%)':>12} {'影响方向':>10}")
print("-"*55)

effects_display = [
    ('碳强度效应', 'delta_CC_m', '碳强度'),
    ('能源强度效应', 'delta_CC_s', '能源强度'),
    ('人均消费效应', 'delta_CC_g', '人均消费'),
    ('人口规模效应', 'delta_CC_p', '人口规模')
]

for name, key, short_name in effects_display:
    value = direct[key]
    contrib = direct_contribution[key]
    direction = "↑促进" if value > 0 else "↓抑制"
    print(f"{name:<15} {value:>15,.2f} {contrib:>11.2f}% {direction:>10}")

print("-"*55)
sum_direct = sum([direct['delta_CC_m'], direct['delta_CC_s'], 
                  direct['delta_CC_g'], direct['delta_CC_p']])
print(f"{'合计':<15} {sum_direct:>15,.2f} {100.0:>11.2f}%")
print(f"{'实际变化':<15} {total_change:>15,.2f}")
print(f"{'分解误差':<15} {abs(sum_direct - total_change):>15,.2f}")

# ===========================
# 方法2：逐年分解累加
# ===========================

print("\n" + "="*70)
print("方法2：逐年分解累加法")
print("="*70)
print("说明：逐年分解后累加，捕捉年度波动的累积效应")

# 执行逐年分解
yearly_decomposition = []
for year in range(1997, 2021):
    yearly_result = lmdi_additive_decomposition(data, year, year+1)
    yearly_decomposition.append({
        'period': f"{year}-{year+1}",
        'delta_CC_m': yearly_result['delta_CC_m'],
        'delta_CC_s': yearly_result['delta_CC_s'],
        'delta_CC_g': yearly_result['delta_CC_g'],
        'delta_CC_p': yearly_result['delta_CC_p'],
        'delta_CC_total': yearly_result['delta_CC_total']
    })

yearly_df = pd.DataFrame(yearly_decomposition)

# 计算累加结果
cumulative = {
    'delta_CC_m': yearly_df['delta_CC_m'].sum(),
    'delta_CC_s': yearly_df['delta_CC_s'].sum(),
    'delta_CC_g': yearly_df['delta_CC_g'].sum(),
    'delta_CC_p': yearly_df['delta_CC_p'].sum(),
    'delta_CC_total': yearly_df['delta_CC_total'].sum()
}

# 计算贡献率
cumulative_contribution = {}
for key in ['delta_CC_m', 'delta_CC_s', 'delta_CC_g', 'delta_CC_p']:
    cumulative_contribution[key] = (cumulative[key] / abs(total_change)) * 100

# 输出结果表格
print(f"\n{'效应类型':<15} {'贡献量(万吨CO2)':>15} {'贡献率(%)':>12} {'影响方向':>10}")
print("-"*55)

for name, key, short_name in effects_display:
    value = cumulative[key]
    contrib = cumulative_contribution[key]
    direction = "↑促进" if value > 0 else "↓抑制"
    print(f"{name:<15} {value:>15,.2f} {contrib:>11.2f}% {direction:>10}")

print("-"*55)
sum_cumulative = sum([cumulative['delta_CC_m'], cumulative['delta_CC_s'], 
                      cumulative['delta_CC_g'], cumulative['delta_CC_p']])
print(f"{'合计':<15} {sum_cumulative:>15,.2f} {100.0:>11.2f}%")
print(f"{'实际变化':<15} {total_change:>15,.2f}")
print(f"{'分解误差':<15} {abs(sum_cumulative - total_change):>15,.2f}")

# ===========================
# 方法对比分析
# ===========================

print("\n" + "="*70)
print("两种方法对比分析")
print("="*70)

# 创建对比表格
comparison_df = pd.DataFrame({
    '效应': ['碳强度', '能源强度', '人均消费', '人口规模', '合计'],
    '直接分解(万吨)': [
        direct['delta_CC_m'],
        direct['delta_CC_s'],
        direct['delta_CC_g'],
        direct['delta_CC_p'],
        sum_direct
    ],
    '逐年累加(万吨)': [
        cumulative['delta_CC_m'],
        cumulative['delta_CC_s'],
        cumulative['delta_CC_g'],
        cumulative['delta_CC_p'],
        sum_cumulative
    ]
})

# 计算差异
comparison_df['绝对差异'] = comparison_df['逐年累加(万吨)'] - comparison_df['直接分解(万吨)']
comparison_df['相对差异(%)'] = np.where(
    comparison_df['直接分解(万吨)'].abs() > 1e-10,
    (comparison_df['绝对差异'] / comparison_df['直接分解(万吨)'].abs()) * 100,
    0
)

# 格式化输出
print("\n方法差异对比表：")
print("-"*70)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
print(comparison_df.to_string(index=False))

# ===========================
# 关键发现总结
# ===========================

print("\n" + "="*70)
print("关键发现与洞察")
print("="*70)

# 1. 主要驱动因素分析
print("\n【1. 碳排放变化的主要驱动因素】")
print("-"*50)

effects_sorted = sorted([
    ('人均消费', direct['delta_CC_g'], direct_contribution['delta_CC_g']),
    ('人口规模', direct['delta_CC_p'], direct_contribution['delta_CC_p']),
    ('碳强度', direct['delta_CC_m'], direct_contribution['delta_CC_m']),
    ('能源强度', direct['delta_CC_s'], direct_contribution['delta_CC_s'])
], key=lambda x: abs(x[1]), reverse=True)

for i, (name, value, contrib) in enumerate(effects_sorted, 1):
    if value > 0:
        print(f"  {i}. {name}效应: +{value:,.2f} 万吨 (贡献率 {contrib:+.2f}%) - 促进排放增长")
    else:
        print(f"  {i}. {name}效应: {value:,.2f} 万吨 (贡献率 {contrib:+.2f}%) - 抑制排放增长")

# 2. 净效应分析
print("\n【2. 净效应分析】")
print("-"*50)

positive_effects = sum([v for v in [direct['delta_CC_m'], direct['delta_CC_s'], 
                                    direct['delta_CC_g'], direct['delta_CC_p']] if v > 0])
negative_effects = sum([v for v in [direct['delta_CC_m'], direct['delta_CC_s'], 
                                    direct['delta_CC_g'], direct['delta_CC_p']] if v < 0])

print(f"  • 促进排放的效应总和: +{positive_effects:,.2f} 万吨CO2")
print(f"  • 抑制排放的效应总和: {negative_effects:,.2f} 万吨CO2")
print(f"  • 净增长: {positive_effects + negative_effects:,.2f} 万吨CO2")
print(f"  • 抵消比例: {abs(negative_effects/positive_effects)*100:.2f}%")

# 3. 方法差异分析
print("\n【3. 方法差异说明】")
print("-"*50)

avg_diff = comparison_df['相对差异(%)'].iloc[:-1].abs().mean()  # 排除总计行
max_diff_idx = comparison_df['相对差异(%)'].iloc[:-1].abs().idxmax()
max_diff_effect = comparison_df.loc[max_diff_idx, '效应']
max_diff_value = comparison_df.loc[max_diff_idx, '相对差异(%)']

print(f"  • 两种方法的平均相对差异: {avg_diff:.2f}%")
print(f"  • 最大差异出现在: {max_diff_effect}效应 ({max_diff_value:.2f}%)")
print(f"  • 差异原因：")
print(f"    - 直接分解：使用期初和期末的对数平均权重")
print(f"    - 逐年累加：每年使用不同的权重函数")
print(f"    - 逐年分解能够捕捉路径依赖性和年度波动")

# 4. 政策启示
print("\n【4. 政策启示】")
print("-"*50)

if direct['delta_CC_g'] > 0 and direct['delta_CC_p'] > 0:
    print("  • 经济增长和人口增长是碳排放增长的主要驱动力")
if direct['delta_CC_m'] < 0:
    print("  • 碳强度改善（能源结构优化）有效抑制了排放增长")
if direct['delta_CC_s'] < 0:
    print("  • 能源效率提升对减排做出了重要贡献")
    
print("\n建议重点关注：")
# 找出抑制效应最弱的因素
weakest_mitigation = min([(name, value) for name, value, _ in effects_sorted if value < 0], 
                         key=lambda x: abs(x[1]), default=None)
if weakest_mitigation:
    print(f"  • 强化{weakest_mitigation[0]}的改善力度")
    
print("\n" + "="*70)
print("分析完成")
print("="*70)

LMDI分解方法对比分析：1997-2021年

【基础信息】
--------------------------------------------------
• 1997年碳排放量: 3,014.46 万吨CO2
• 2021年碳排放量: 10,156.23 万吨CO2
• 总变化量: 7,141.76 万吨CO2
• 累计增长率: 236.92%
• 年均增长率: 5.19%

方法1：直接分解法（1997→2021）
说明：直接计算整个时期的分解效应，反映长期累积影响

效应类型                 贡献量(万吨CO2)       贡献率(%)       影响方向
-------------------------------------------------------
碳强度效应                   -814.05      -11.40%        ↓抑制
能源强度效应                -3,696.94      -51.77%        ↓抑制
人均消费效应                10,868.76      152.19%        ↑促进
人口规模效应                   784.00       10.98%        ↑促进
-------------------------------------------------------
合计                     7,141.76      100.00%
实际变化                   7,141.76
分解误差                       0.00

方法2：逐年分解累加法
说明：逐年分解后累加，捕捉年度波动的累积效应

效应类型                 贡献量(万吨CO2)       贡献率(%)       影响方向
-------------------------------------------------------
碳强度效应                   -496.75       -6.96%        ↓抑制
能源强度效应                -4,527.72      -63.40%      